<a href="https://colab.research.google.com/github/BrainConnection/Qiskit/blob/main/Open%20Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setting

In [2]:
!pip install pennylane

import pennylane as qml
import pennylane.numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 56.3 MB/s eta 0:00:00


# 3. Preparing for Battle

## 1) Real Data Amplitude Embedding (Method 1)

In [20]:
def preparing_for_battle_1(x):
  """
  Conduct Amplitude Embdding

  Args
    - x (numpy.tensor): a length-N vector

  Returns
    - state (qml.measurements.StateMP): Embedded state
  """

  x = x/np.linalg.norm(x)
  N = np.shape(x)[0]
  num_qubits = (int)(np.log2(N))


  def binary_encode(num):

    lst = []

    for i in range(num_qubits):
      lst.append(num % 2)
      num = num // 2

    lst.reverse()

    return lst


  dev = qml.device("default.qubit", wires=range(num_qubits+1))
  @qml.qnode(dev)
  def amplitude_embedding():

    for i in range(num_qubits):
      qml.Hadamard(i)

    for i in range(N):
      U = np.array([[x[i], -np.sqrt(1-x[i]**2)], [np.sqrt(1-x[i]**2), x[i]]])
      qml.ControlledQubitUnitary(U, control_wires=range(num_qubits), wires=num_qubits, control_values=binary_encode(i))

    qml.measure(num_qubits, postselect=0)

    return qml.probs(range(num_qubits))


  print(qml.draw(amplitude_embedding)())
  print()

  return np.sqrt(amplitude_embedding())


In [21]:
print(preparing_for_battle_1(np.array([1,2,3,4,5,6,7,8])))

0: ──H─╭○─────╭○─────╭○─────╭○─────╭●─────╭●─────╭●─────╭●───────────┤ ╭Probs
1: ──H─├○─────├○─────├●─────├●─────├○─────├○─────├●─────├●───────────┤ ├Probs
2: ──H─├○─────├●─────├○─────├●─────├○─────├●─────├○─────├●───────────┤ ╰Probs
3: ────╰U(M0)─╰U(M1)─╰U(M2)─╰U(M3)─╰U(M4)─╰U(M5)─╰U(M6)─╰U(M7)──┤↗₀├─┤       

M0 = 
[[ 0.070014   -0.99754601]
 [ 0.99754601  0.070014  ]]
M1 = 
[[ 0.14002801 -0.99014754]
 [ 0.99014754  0.14002801]]
M2 = 
[[ 0.21004201 -0.97769236]
 [ 0.97769236  0.21004201]]
M3 = 
[[ 0.28005602 -0.95998366]
 [ 0.95998366  0.28005602]]
M4 = 
[[ 0.35007002 -0.93672353]
 [ 0.93672353  0.35007002]]
M5 = 
[[ 0.42008403 -0.90748521]
 [ 0.90748521  0.42008403]]
M6 = 
[[ 0.49009803 -0.87166732]
 [ 0.87166732  0.49009803]]
M7 = 
[[ 0.56011203 -0.82841687]
 [ 0.82841687  0.56011203]]

[0.070014   0.14002801 0.21004201 0.28005602 0.35007002 0.42008403
 0.49009803 0.56011203]


## 2) Real Data Amplitude Embedding (Method 2)

In [31]:
def preparing_for_battle_2(x):
  """
  Conduct Amplitude Embdding

  Args
    - x (numpy.tensor): a length-N vector

  Returns
    - state (qml.measurements.StateMP): Embedded state
  """

  x = x/np.linalg.norm(x)
  N = np.shape(x)[0]
  num_qubits = (int)(np.log2(N))


  def theta_generate(x):
    y = []

    for i in range(num_qubits):
      lst = []

      for j in range(2**(i+1)):
        sum = 0

        num = (int)(N/2**(i+1))
        for k in range(j*num, (j+1)*num, 1):
          sum = sum + x[k]**2

        lst.append(np.sqrt(sum))

      y.append(lst)

    theta = []

    for i in range(num_qubits):
      lst = []

      if i==0:
        lst.append(2*np.arccos(y[i][0]))

      else:
        for j in range(2**i):
          lst.append(2*np.arccos(y[i][2*j]/y[i-1][j]))

      theta.append(lst)

    return theta


  def control_generate(num_qubits):
    control = []

    for i in range(1, num_qubits):
      lst_1 = []

      for j in range(2**i):
        lst_2 = []

        num = j
        for k in range(i):
          lst_2.append(num % 2)
          num = num // 2

        lst_2.reverse()
        lst_1.append(lst_2)

      control.append(lst_1)

    return control


  dev = qml.device("default.qubit", wires=range(num_qubits+1))
  @qml.qnode(dev)
  def amplitude_embedding():

    theta = theta_generate(x)
    control_bit = control_generate(num_qubits)

    qml.RY(theta[0][0],0)

    for i in range(1, num_qubits, 1):
      for j in range(2**i):
        U = np.array([[np.cos(theta[i][j]/2), -np.sin(theta[i][j]/2)], [np.sin(theta[i][j]/2), np.cos(theta[i][j]/2)]])
        qml.ControlledQubitUnitary(U, control_wires=range(i), wires=[i], control_values=control_bit[i-1][j])


    return qml.probs(range(num_qubits))


  print(qml.draw(amplitude_embedding)())
  print()

  return np.sqrt(amplitude_embedding())

In [35]:
print(preparing_for_battle_2(np.array([1,2,3,4,5,6,7,8])))

0: ──RY(2.35)─╭○─────╭●─────╭○─────╭○─────╭●─────╭●─────┤ ╭Probs
1: ───────────╰U(M0)─╰U(M1)─├○─────├●─────├○─────├●─────┤ ├Probs
2: ─────────────────────────╰U(M2)─╰U(M3)─╰U(M4)─╰U(M5)─┤ ╰Probs

M0 = 
[[ 0.40824829 -0.91287093]
 [ 0.91287093  0.40824829]]
M1 = 
[[ 0.5920935  -0.80586927]
 [ 0.80586927  0.5920935 ]]
M2 = 
[[ 0.4472136  -0.89442719]
 [ 0.89442719  0.4472136 ]]
M3 = 
[[ 0.6 -0.8]
 [ 0.8  0.6]]
M4 = 
[[ 0.6401844  -0.76822128]
 [ 0.76822128  0.6401844 ]]
M5 = 
[[ 0.65850461 -0.75257669]
 [ 0.75257669  0.65850461]]

[0.070014   0.14002801 0.21004201 0.28005602 0.35007002 0.42008403
 0.49009803 0.56011203]


## 3) Complex Data Amplitude Embedding (Method 1)